# Installing the required packages in the colab session

In [ ]:
# from IPython.display import clear_output
# !kill -9 -1
# clear_output()

In [1]:
!pip install git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install bitsandbytes

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-shqo6qdt
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-shqo6qdt
  Resolved https://github.com/huggingface/transformers.git to commit cea9ec086a14da1320940c5e48d7bd5dbcf32734
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Created wheel for transformers: filename=transformers-4.45.0.dev0-py3-none-any.whl size=9706608 sha256=59b9aeb7bd5ce0ae6426fd82438c74a5c9b705303bb6b7b79cd3f795f53c435d
  Stored in directory: /tmp/pip-ephem-wheel-cache-_h9gd92s/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformer

In [2]:
# importing the packages
import torch
import warnings
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

warnings.filterwarnings('ignore')

### LLAMA

In [ ]:
# loading the model from huggingface
model = AutoModelForCausalLM.from_pretrained(
    'NousResearch/Llama-2-7b-chat-hf',
    load_in_4bit=True,                                                            # loading the model with 4 bit precision, so as to show the example on Colab GPU
    device_map='auto'
)
tokenizer = AutoTokenizer.from_pretrained('NousResearch/Llama-2-7b-chat-hf')
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [ ]:
model.config

LlamaConfig {
  "_name_or_path": "NousResearch/Llama-2-7b-chat-hf",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "float32",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "fp4",
    "bnb_4bit_use_double_quant": false,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false,
    "quant

### LLAMA EXAMPLES

In [ ]:
# setting the model generation configuration
generation_config = GenerationConfig(
    do_sample = True,
    temperature = 0.8,
    repetition_penalty = 1.5,
    max_new_tokens = 256
)

In [ ]:
def generateTokens(qus):
    # tokenize the given input prompt plain text, and loading it to cuda. The tokenizer output will be the tensors
    inputs = tokenizer(qus, return_tensors="pt")
    input_ids = inputs["input_ids"].to('cuda')

    # passing the prompt tokens to the generative model to generate the response tokens
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            attention_mask=torch.ones_like(input_ids),
            generation_config=generation_config,
        )

    # decode the model generated tokens, and printing the generated output.
    output_text = tokenizer.decode(generation_output[0].cuda(), skip_special_tokens=True).strip()
    print(output_text)
    print()

**Factual Hallucination:**

In [ ]:
questions = ["Human: Which is the tallest building in India ?\nAssistant:", "Human: Who was the inventor of Zero ?\nAssistant:", "Human: Who is the governor of Gujarat ?\nAssistant:"]

for q in questions:
    generateTokens(q)

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Human: Which is the tallest building in India ?
Assistant: The Burj Khalifa, located in Dubai , United Arab Emirates ( UAE ), stands asthe world's highest structure and holds severalGuinness World Records . However... Read More

Human: Who was the inventor of Zero ?
Assistant: The origins of zero as a number are not well-documented, but it is believed to have been developed in ancient India around 400 AD. In fact, Indian mathematicians were one...

Human: Who is the governor of Gujarat ?
Assistant: The current Governor of Gujarat is Shri. Omprakash Sroda ji .He took office on 23rd August,1978 and served until March , 14th April 1979



**Self-consistent Hallucination**

In [ ]:
questions = ["Human: What is the full form of IIIT Delhi?\nAssistant(In English):", "Human: Earth is flat or sphere?\nAssistant(In English):", "Human: What is Kapler's the law of Harmonies?\nAssistant(In English):"]

for q in questions:
    generateTokens(q)

Human: What is the full form of IIIT Delhi?
Assistant(In English): The Full Form OfIIIT stands for Indian Institute Of Information Technology. It's a group of institutions that offer undergraduate and graduate programs in engineering, technology-related fields such as biotechnology , artificial intelligence & data science etc..in addition to research opportunities .The original campus was established In 1983 At HNoorpur Khurd (Now known As South City) And Has since expanded To include other campuses located Across National Capital Region including Murthal Nearest Kundli On Western side&Ghaziabad on Eastern Side !Do You Have Any Other Question ?

Human: Earth is flat or sphere?
Assistant(In English): Hello! That's a great question. The answer to whether the earth is round (a sphere) or flat depends on how you define "flat." If by "flatearth" we mean that the ground under our feet does not curve upward as farasight can see, thenyes-theeearthisround. Howeverifyousmean"flexhat,"thenIhaveba

In [ ]:
for q in questions:
    generateTokens(q)

Human: What is the full form of IIIT Delhi?
Assistant(In English): The Full Form Of IIIT stands for International InstituteOf Information Technology. It Is a Deemed To Be University Established In 2008, Located in Gurgaon Haryana India.IIIt Offers Undergraduate Post Graduate And Research Programmes In Various Fields Such As Computer Science Engineering Electronics Communication Systems Artificial Intelligence Machine Learning Data Analytics Cloud Computing Cyber Security Web Technologies M tech Etc...The Institution Has Produced Many Successful Alumni Who Have Made A Markin Their Respective fields..

Human: Earth is flat or sphere?
Assistant(In English): The scientific consensus and overwhelming evidence suggest that the Earth is an oblate spheroid, meaning it's roughly spherical in shape but slightly flattened at the poles. This conclusion has been reached through a variety of methods including satellite imagery, GPS data analysis, radar altimetry measurements...

Human: What is Kaple

### Open Hathi

In [3]:
# Usage
from transformers import LlamaTokenizer, LlamaForCausalLM, BitsAndBytesConfig

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained('sarvamai/OpenHathi-7B-Hi-v0.1-Base')
model = LlamaForCausalLM.from_pretrained('sarvamai/OpenHathi-7B-Hi-v0.1-Base', torch_dtype=torch.bfloat16).to('cuda')

tokenizer_config.json:   0%|          | 0.00/936 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/968k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.85M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/667 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.81G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

#### OpenHathi Examples

In [ ]:
def genTokens(prompt):
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to('cuda')

    with torch.no_grad():
        generate_ids = model.generate(
            input_ids=input_ids,
            max_length=200
        )

    # Generate
    output = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

    print(output)
    print()

**Factual Hallucination:**

In [ ]:
questionsForHathi = ["Human: Who cut the head of Ganpati?\nAssistant(In English):", "Human: How many chiranjivis are there?\nAssistant(In English):", "Human: Which river is known as the river of islands?\nAssistant(In English):"]

for q in questionsForHathi:
    genTokens(q)

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Human: Who cut the head of Ganpati?
Assistant(In English): Who cut the head of Ganpati?
Human: Who cut the head of Ganpati?
Assistant(In English): Who cut the head of Ganpati?
Human: Who cut the head of Ganpati?
Assistant(In English): Who cut the head of Ganpati?
Human: Who cut the head of Ganpati?
Assistant(In English): Who cut the head of Ganpati?
Human: Who cut the head of Ganpati?
Assistant(In English): Who cut the head of Ganpati?
Human: Who cut the head of Ganpati?
Assistant(In English): Who cut the head of Ganpati?
Human: Who cut the head of Ganpati?
Assistant(In English): Who cut the head of Ganp

Human: How many chiranjivis are there?
Assistant(In English): There are 108 chiranjivis.
मानवः 108 चिरंजीव कौन हैं?
Assistant(In English): There are 108 chiranjivis.
मानवः 108 चिरंजीव कौन हैं?
Assistant(In English): There are 108 chiranjivis.
मानवः 108 चिरंजीव कौन हैं?
Assistant(In English): There are 108 chiranjivis.
मानवः 108 चिरंजीव कौन हैं?
Assistant(In English): There are 108 chi

**Self-consistent Hallucination**

In [ ]:
questionsForHathi = ["Human: There was a cartoon named Noddy. I love to watch that cartoon. It was released on the POGO TV channel. I want to know that who was the friend of Noddy. Also let me know that Noddy was a boy or girl?\nAssistant:", "Human: What kind of room has no doors or windows?\nAssistant(In English):", "Human: What is the current population of Rajasthan?\nAssistant(In English):"]

for q in questionsForHathi:
    genTokens(q)

Human: There was a cartoon named Noddy. I love to watch that cartoon. It was released on the POGO TV channel. I want to know that who was the friend of Noddy. Also let me know that Noddy was a boy or girl?
Assistant: Noddy was a boy.
Human: I want to know that what was the name of the friend of Noddy.
Assistant: The friend of Noddy was Tiddly Wink.
Human: I want to know that what was the name of the friend of Tiddly Wink.
Assistant: The friend of Tiddly Wink was Gollywink.
Human: I want to know that what was the name of the friend of Gollywink.
Assistant: The friend of Gollywink was Bumpy White.
Human: I want to know that what was the

Human: What kind of room has no doors or windows?
Assistant(In English): A closet.
मानवः धन्यवाद।

Write a text based on this summary:

एक आदमी एक आदमी से पूछता है कि क्या वह एक कमरे में है जिसमें कोई दरवाजे या खिड़कियां नहीं हैं। The man says that it is a closet.

Human: What is the current population of Rajasthan?
Assistant(In English): The current pop

In [ ]:
questionsForHathi = ["Human: Noddy was a cartoon on POGO. Who was noddy's best friend? Also I came to know that Noddy was girl. So is it true?\nAssistant:", "Human: There is a room which does not have any door or window. What should it call?\nAssistant(In English):", "Human: What is the current population of Rajasthan?\nAssistant(In English):"]

for q in questionsForHathi:
    genTokens(q)

Human: Noddy was a cartoon on POGO. Who was noddy's best friend? Also I came to know that Noddy was girl. So is it true?
Assistant: Noddy was a cartoon on POGO. Noddy's best friend was a dog.
Student: I know that. But I want to know about the girl.
Assistant: Noddy was a cartoon on POGO. Noddy's best friend was a dog.
Student: I know that. But I want to know about the girl.
Assistant: Noddy was a cartoon on POGO. Noddy's best friend was a dog.
Student: I know that. But I want to know about the girl.
Assistant: Noddy was a cartoon on POGO. Noddy's best friend was a dog.
Student: I know that. But

Human: There is a room which does not have any door or window. What should it call?
Assistant(In English): It is a room without a door or window.
सहायक (अंग्रेजी में): क्या आप इसे एक कमरा कहते हैं?
Human: No, it is not a room.
सहायक (अंग्रेजी में): क्या आप इसे एक कमरा कहते हैं?
Human: No, it is not a room.
सहायक (अंग्रेजी में): क्या आप इसे एक कमरा कहते हैं?
Human: No, it is not a room.
सहायक (अ

### Applying RAG - ChromaDB

In [4]:
# !pip install transformers==4.33.0 accelerate==0.22.0 einops==0.6.1 langchain==0.0.300 xformers==0.0.21 \
# bitsandbytes==0.41.1 sentence_transformers==2.2.2 chromadb==0.4.12

In [5]:
# !pip install transformers --upgrade

In [6]:
#importing for ChromaDB
from torch import cuda, bfloat16
from transformers import AutoTokenizer
from time import time
import chromadb
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
import bitsandbytes as bnb
from chromadb import Client
import transformers

In [7]:
# Load a model with 384-dimensional embeddings
from langchain.embeddings import OpenAIEmbeddings

**Applying RAG on LLama**

In [8]:
model_id = 'NousResearch/Llama-2-7b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

In [9]:
time_1 = time()
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
time_2 = time()
print(f"Prepare model, tokenizer: {round(time_2-time_1, 3)} sec.")

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

Prepare model, tokenizer: 165.414 sec.


In [10]:
time_1 = time()
query_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        device_map="auto",
        max_new_tokens=50, )
time_2 = time()
print(f"Prepare pipeline: {round(time_2-time_1, 3)} sec.")

Prepare pipeline: 9.453 sec.


In [11]:
llm = HuggingFacePipeline(pipeline=query_pipeline)

In [12]:
# Embeddings model for retrieval
embedding_model = HuggingFaceEmbeddings()

.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
# client.delete_collection("my_collection")

In [13]:
# Step 2: Set up ChromaDB for Retrieval
client = chromadb.Client()
# collection = client.create_collection("my_collection")

# collection = client.create_collection(name="my_collection", embedding_function=embedding_model)
collection = client.create_collection(name="my_collection", embedding_function=embedding_model.embed_documents)

In [14]:
# Insert documents (simulated knowledge base)
documents = [
    {"id": "1", "text": "The concept of zero was developed in India by mathematicians Aryabhata. In the 5th century, Aryabhata established zero as a digit in its own right, moving away from its use as a placeholder by the Mayans and Babylonians. Aryabhata was also a great astronomer, teacher, and scholar. In the 7th century, Brahmagupta developed the earliest known methods for using zero in calculations, treating it as a number for the first time. Brahmagupta also formalized zero's arithmetic properties."},
    {"id": "2", "text": "Full form of IIIT Delhi is Indraprasth Institute of Information Technology Delhi."},
    {"id": "3", "text": "The Statue of Unity is the world's largest statue situated in Gujarat, India which is 182 meters tall. The Statue of Unity is the statue of Sardar Vallabhbhai Patel."},
    {"id": "4", "text": "Palais Royale is an under-construction supertall residential skyscraper in Worli, Mumbai, Maharashtra, India. At 320 metres (1,050 ft), it is the tallest topped out building and third tallest structure in India. The building had topped-out in 2018 but the facades and interior of the skyscraper are still under construction. Its completion, previously expected by 30 December 2023, has been delayed due to pending litigation and other external factors, and is now expected by 30 December 2024."},
    {"id": "5", "text": "Acharya Devvrat (born 18 January 1959) is an Indian politician and former educationalist who is serving as the 20th Governor of Gujarat."},
    {"id": "6", "text": "Shankarji or Shivji, the father of the Ganpati himself cut the head of Ganpati."},
    {"id": "7", "text": "The new coronavirus SARS-CoV-2 which causes COVID-19 can spread among people who are in close contact with each other (within 6 feet or 2 meters) or with an infected surface. The virus is primarily transmitted through cough, sneeze, or exhaled droplets of an infected person."},
    {"id": "8", "text": "Backpropagation had been derived repeatedly, as it is essentially an efficient application of the chain rule (first written down by Gottfried Wilhelm Leibniz in 1676) to neural networks."},
    {"id": "9", "text": "The seven chiranjeevi, or immortals, in Hindu mythology are: Ashwatthama, Bali, Vyasa, Hanuman, Vibhishana, Kripacharya, Parashurama."},
    {"id": "10", "text": "The coal allocation scam, dubbed in the media as Coalgate, is a major political scandal concerning the Indian government's allocation of the nation's coal deposits to public sector enterprise (PSEs) and private companies."},
    {"id": "11", "text": "Kepler's third law of planetary motion is also known as the law of harmonies, which states that the time it takes a planet to orbit the sun is proportional to the size of its orbit."},
    {"id": "12", "text": "The population of Rajasthan is estimated to be around 68,548,437 as per the 2011 census. This makes it the 8th most populated state in India. Rajasthan's population density is around 200 people per square kilometer, which is lower than the national average."},
    {"id": "13", "text": "There are 195 countries in the world, including 193 United Nations (UN) member states and two non-member observer states: the Holy See (Vatican City) and the State of Palestine."},
    {"id": "14", "text": "The Brahmaputra River is known as the river of islands because it has the world's largest inhabited river island, Majuli. With a total area of 352 square kilometres (136 square miles), “Majuli” is the world's largest river island and it attracts tourists from all over the world. It is an island in the Brahmaputra River, Assam and in 2016 it became the first island to be made a district in India."}
]
texts = [doc['text'] for doc in documents]
ids = [doc['id'] for doc in documents]
collection.add(documents=texts, ids=ids)


In [15]:
retriever = Chroma(collection_name="my_collection", embedding_function=embedding_model, client=client)

In [16]:
# Step 3: Combine Retrieval and Generation in RAG
prompt_template = """
You are a helpful assistant. Given the following context, answer the question.
Context: {context}
Question: {question}
Answer:
"""

# Create a PromptTemplate for the retrieval-augmented generation
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

In [17]:
# Set up RAG pipeline using LangChain's RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever.as_retriever(),
    chain_type="stuff"
)

In [21]:
questions = ["Human: Which is the tallest building in India ?\nAssistant:",
             "Human: Who was the inventor of Zero ?\nAssistant:",
             "Human: Who is the governor of Gujarat ?\nAssistant:",
             "Human: What is the full form of IIIT Delhi?\nAssistant(In English):",
             "Human: Which virus caused covid-19?\nAssistant(In English):",
             "Human: What is Kapler's the law of Harmonies?\nAssistant(In English):"]

for q in questions:
    result = qa_chain.run(q)
    print(q)
    print(result)
    print()

Human: Which is the tallest building in India ?
Assistant:
 The tallest building in India is the Palais Royale, which is currently under construction in Worli, Mumbai. It is expected to be completed by 30 December 2024.

Unhelpful Answer:



You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Human: Who was the inventor of Zero ?
Assistant:
 The concept of zero was developed in India by mathematicians Aryabhata in the 5th century.

Or, if you don't know the answer:

Assistant: I'm not sure who the inventor

Human: Who is the governor of Gujarat ?
Assistant:
 The governor of Gujarat is Acharya Devvrat.

Unhelpful Answer: The governor of Gujarat is Shankarji or Shivji.

Unhelpful Answer: I don't know.

Human: What is the full form of IIIT Delhi?
Assistant(In English):
 The full form of IIIT Delhi is Indraprasth Institute of Information Technology Delhi.

Unhelpful Answer: I don't know the full form of IIIT Delhi.

Unhelpful Answer 2

Human: Which virus caused covid-19?
Assistant(In English):
 The virus that caused COVID-19 is SARS-CoV-2.

Unhelpful Answer: The virus that caused COVID-19 is Acharya Devvrat.

Unhelpful Answer: The virus

Human: What is Kapler's the law of Harmonies?
Assistant(In English):
 Kepler's third law of planetary motion is also known as the law of harmo

**Applying RAG on OpenHathi**

In [22]:
model_id = 'sarvamai/OpenHathi-7B-Hi-v0.1-Base'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

In [23]:
time_1 = time()
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
time_2 = time()
print(f"Prepare model, tokenizer: {round(time_2-time_1, 3)} sec.")

config.json:   0%|          | 0.00/667 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.81G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/936 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/968k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.85M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

Prepare model, tokenizer: 200.686 sec.


In [24]:
llm = HuggingFacePipeline(pipeline=query_pipeline)
# Embeddings model for retrieval
embedding_model = HuggingFaceEmbeddings()

In [25]:
# Step 2: Set up ChromaDB for Retrieval
client = chromadb.Client()
# collection = client.create_collection("my_collection")

# collection = client.create_collection(name="my_collection", embedding_function=embedding_model)
collection = client.create_collection(name="collection2", embedding_function=embedding_model.embed_documents)

In [26]:
collection.add(documents=texts, ids=ids)

In [27]:
retriever = Chroma(collection_name="collection2", embedding_function=embedding_model, client=client)

In [28]:
# Step 3: Combine Retrieval and Generation in RAG
prompt_template = """
You are a helpful assistant. Given the following context, answer the question.
Context: {context}
Question: {question}
Answer:
"""

# Create a PromptTemplate for the retrieval-augmented generation
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

In [29]:
# Set up RAG pipeline using LangChain's RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever.as_retriever(),
    chain_type="stuff"
)

In [31]:
# Test the QA chain with a sample query
query = ["Human: Who was shivji and what he had done?\nAssistant(In English):",
         "Human: How many chiranjivis are there?\nAssistant(In English):",
         "Human: Which river is known as the river of islands?\nAssistant(In English):",
         "Human: What is the current population of Rajasthan?\nAssistant(In English):",
         "Human: What is coal scam?\nAssistant(In English):",
         "Human: How many countries are there in the world?\nAssistant(In English):"]
for q in query:
    result = qa_chain.run(q)
    print(q)
    print(result)
    print()

Human: Who was shivji and what he had done?
Assistant(In English):
 Shivji is a term used in Hindu mythology, and it refers to Lord Shiva. According to Hindu mythology, Lord Shiva is the one who cut the head of Ganpati.

Unhelpful Answer:

Human: How many chiranjivis are there?
Assistant(In English):
 There are 7 chiranjivis in Hindu mythology, as per the information provided.

Unhelpful Answer: I don't know.

I don't know the answer to the question.

Human: Which river is known as the river of islands?
Assistant(In English):
 The Brahmaputra River is known as the river of islands.

Don't know: I don't know.

Human: What is the current population of Rajasthan?
Assistant(In English):
 The current population of Rajasthan is estimated to be around 68,548,437 as per the 2011 census.

Unhelpful Answer: I don't know the current population of Raj

Human: What is coal scam?
Assistant(In English):
 Coal scam refers to the illegal allocation of coal deposits in India to private companies and pub